In [1]:
import pandas as pd
import numpy as np
df= pd.read_csv('EW_csv.csv',encoding='CP949')


# 1. EW등급 20~30 업종분석

In [2]:
temp = (df.대표자연령구간 == '20대') | (df.대표자연령구간 == '30대')
df1=df[temp]

In [3]:
df1['기준일자']=df1['기준일자'].astype('str')
df1['기준일자']=pd.to_datetime(df1['기준일자'])

<ipython-input-3-46ca327bc4be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['기준일자']=df1['기준일자'].astype('str')
<ipython-input-3-46ca327bc4be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['기준일자']=pd.to_datetime(df1['기준일자'])


In [4]:
test1 = df1.groupby(['업종중분류','EW등급'])['EW등급'].count()
test1=pd.DataFrame(test1)
test1=test1.unstack()
test1 = test1.fillna(0)
test1=test1.reset_index()
test1

업종중분류    EW등급                                         \
EW등급                               관심     관찰1     관찰2     관찰3    부도       정상   
0                   1차 금속 제조업   366.0   309.0  1161.0   822.0  19.0  17287.0   
1                   가구 내 고용활동     6.0    38.0     9.0     0.0   0.0     24.0   
2                      가구 제조업   231.0   325.0   719.0   508.0   0.0  13322.0   
3             가죽, 가방 및 신발 제조업   280.0   279.0   679.0   558.0   0.0  11264.0   
4               개인 및 소비용품 수리업   185.0   637.0  1477.0   251.0   0.0  10495.0   
..                        ...     ...     ...     ...     ...   ...      ...   
71            하수, 폐수 및 분뇨 처리업    17.0    11.0    46.0    33.0   0.0   1786.0   
72                     항공 운송업     1.0     0.0     1.0     0.0   0.0    346.0   
73                    협회 및 단체   179.0   853.0  1445.0   262.0   0.0  35554.0   
74    화학물질 및 화학제품 제조업; 의약품 제외  1592.0  1709.0  3356.0  1740.0  40.0  59413.0   
75                환경 정화 및 복원업     6.0    11.0    72.0     6.0   0.0   1185.0   

                     
EW등급      폐업     휴업  
0     3035.0   45.0  
1        0.0    0.0  
2     1965.0   19.0  
3     2135.0   66.0  
4     1454.0   22.0  
..       ...    ...  
71     163.0   25.0  
72      48.0    0.0  
73    1131.0   57.0  
74    6287.0  181.0  
75     735.0   39.0  

[76 rows x 9 columns]

In [5]:
# 합계 컬럼 추가
test1['총합']=test1.sum(axis=1)
#정상률
total1 = test1['총합']
normal1 = test1['EW등급']['정상']
rate1 = (normal1/total1)*100
test1['정상률']=rate1
#부도율 컬럼 추가
budo1=((test1['EW등급']['관심']*0.15)+(test1['EW등급']['관찰1']*0.20)+(test1['EW등급']['관찰2']*0.30)+(test1['EW등급']['관찰3']*0.90)+(test1['EW등급']['부도']*1))
budo_sum1=test1['EW등급']['관심']+test1['EW등급']['관찰1']+test1['EW등급']['관찰2']+test1['EW등급']['관찰3']+test1['EW등급']['부도']+test1['EW등급']['정상']+test1['EW등급']['폐업']
budo_rate1 = (budo1/budo_sum1)*100
test1['부도율']=budo_rate1
test1

업종중분류    EW등급                                         \
EW등급                               관심     관찰1     관찰2     관찰3    부도       정상   
0                   1차 금속 제조업   366.0   309.0  1161.0   822.0  19.0  17287.0   
1                   가구 내 고용활동     6.0    38.0     9.0     0.0   0.0     24.0   
2                      가구 제조업   231.0   325.0   719.0   508.0   0.0  13322.0   
3             가죽, 가방 및 신발 제조업   280.0   279.0   679.0   558.0   0.0  11264.0   
4               개인 및 소비용품 수리업   185.0   637.0  1477.0   251.0   0.0  10495.0   
..                        ...     ...     ...     ...     ...   ...      ...   
71            하수, 폐수 및 분뇨 처리업    17.0    11.0    46.0    33.0   0.0   1786.0   
72                     항공 운송업     1.0     0.0     1.0     0.0   0.0    346.0   
73                    협회 및 단체   179.0   853.0  1445.0   262.0   0.0  35554.0   
74    화학물질 및 화학제품 제조업; 의약품 제외  1592.0  1709.0  3356.0  1740.0  40.0  59413.0   
75                환경 정화 및 복원업     6.0    11.0    72.0     6.0   0.0   1185.0   

                          총합        정상률        부도율  
EW등급      폐업     휴업                                 
0     3035.0   45.0  23044.0  75.017358   5.321101  
1        0.0    0.0     77.0  31.168831  14.545455  
2     1965.0   19.0  17089.0  77.956580   4.525776  
3     2135.0   66.0  15261.0  73.809056   5.289240  
4     1454.0   22.0  14521.0  72.274637   5.684185  
..       ...    ...      ...        ...        ...  
71     163.0   25.0   2081.0  85.824123   2.346790  
72      48.0    0.0    396.0  87.373737   0.113636  
73    1131.0   57.0  39481.0  90.053443   2.198534  
74    6287.0  181.0  74318.0  79.944293   4.307431  
75     735.0   39.0   2054.0  57.692308   1.493797  

[76 rows x 12 columns]

In [6]:
a=test1.sort_values(by=test1.columns[-1],ascending=False).head(10)

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'Malgun Gothic'

In [8]:
a.set_index('업종중분류')

EW등급                                    \
EW등급                               관심      관찰1      관찰2      관찰3     부도   
업종중분류                                                                     
가구 내 고용활동                         6.0     38.0      9.0      0.0    0.0   
석탄, 원유 및 천연가스 광업                  6.0      9.0     35.0     13.0    0.0   
기타 운송장비 제조업                     386.0    703.0   2858.0    822.0    0.0   
보건업                              74.0    184.0    488.0    207.0   27.0   
전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업   1175.0   2090.0  12704.0   2029.0    0.0   
사업지원 서비스업                      2770.0   7297.0  29115.0   3628.0    0.0   
도매 및 상품 중개업                   12437.0  36808.0  81306.0  21533.0  520.0   
소매업; 자동차 제외                    2871.0  13592.0  27833.0   4137.0   34.0   
산업용 기계 및 장비 수리업                  54.0    178.0    428.0    102.0    0.0   
우편 및 통신업                        135.0    671.0   1896.0    335.0    0.0   

                                                               총합        정상률  \
EW등급                                정상       폐업      휴업                        
업종중분류                                                                          
가구 내 고용활동                         24.0      0.0     0.0      77.0  31.168831   
석탄, 원유 및 천연가스 광업                 170.0      0.0     0.0     233.0  72.961373   
기타 운송장비 제조업                    12951.0   3914.0   123.0   21757.0  59.525670   
보건업                             3739.0    531.0    35.0    5285.0  70.747398   
전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업   53531.0  12914.0   191.0   84634.0  63.249994   
사업지원 서비스업                     131595.0  29702.0   811.0  204918.0  64.218370   
도매 및 상품 중개업                   560691.0  90405.0  2236.0  805936.0  69.570164   
소매업; 자동차 제외                   167751.0  26537.0   576.0  243331.0  68.939428   
산업용 기계 및 장비 수리업                 3246.0    241.0     1.0    4250.0  76.376471   
우편 및 통신업                        9612.0   3933.0    24.0   16606.0  57.882693   

                                    부도율  
EW등급                                     
업종중분류                                    
가구 내 고용활동                     14.545455  
석탄, 원유 및 천연가스 광업              10.686695  
기타 운송장비 제조업                    8.300361  
보건업                            7.763810  
전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업   7.379593  
사업지원 서비스업                      6.797709  
도매 및 상품 중개업                    6.659033  
소매업; 자동차 제외                    6.284629  
산업용 기계 및 장비 수리업                6.210873  
우편 및 통신업                       6.179894

In [9]:
test1.sort_values(by=test1.columns[-3],ascending=False).head(10)

업종중분류     EW등급                                              \
EW등급                    관심      관찰1      관찰2      관찰3     부도        정상   
22    도매 및 상품 중개업  12437.0  36808.0  81306.0  21533.0  520.0  560691.0   
60       전문직별 공사업  16360.0   4358.0  15264.0   3439.0   41.0  204780.0   
27           부동산업   5110.0   7635.0  19289.0   3769.0   33.0  216631.0   
36    소매업; 자동차 제외   2871.0  13592.0  27833.0   4137.0   34.0  167751.0   
66            출판업   5151.0   7795.0  14906.0   4692.0    0.0  151736.0   
31      사업지원 서비스업   2770.0   7297.0  29115.0   3628.0    0.0  131595.0   
59        전문 서비스업   2724.0   7616.0  17744.0   3757.0    0.0  138806.0   
63         종합 건설업   8048.0   3377.0  10960.0   3048.0   41.0  112016.0   
41        식료품 제조업   2794.0   2772.0   6645.0   4321.0   11.0  107002.0   
20             농업   1501.0   1545.0   4477.0   1839.0    9.0   76797.0   

                             총합        정상률       부도율  
EW등급       폐업      휴업                                 
22    90405.0  2236.0  805936.0  69.570164  6.659033  
60    26506.0   515.0  271263.0  75.491313  4.077925  
27    18223.0   553.0  271243.0  79.866024  4.250360  
36    26537.0   576.0  243331.0  68.939428  6.284629  
66    27873.0   564.0  212717.0  71.332334  5.197310  
31    29702.0   811.0  204918.0  64.218370  6.797709  
59    18912.0   470.0  190029.0  73.044641  5.611076  
63    18588.0   469.0  156547.0  71.554230  5.096682  
41    14225.0   341.0  138111.0  77.475364  4.984322  
20     5630.0   137.0   91935.0  83.534019  3.857764

# 1-2 중장년층 EW등급 업종분석

In [10]:
temp1_1 = (df.대표자연령구간 == '40대') | (df.대표자연령구간 == '50대')| (df.대표자연령구간 == '60대')
df1_1=df[temp1_1]

In [11]:
test1_1 = df1_1.groupby(['업종중분류','EW등급'])['EW등급'].count()
test1_1=pd.DataFrame(test1_1)
test1_1=test1_1.unstack()
test1_1 = test1_1.fillna(0)
test1_1=test1_1.reset_index()
test1_1

업종중분류    EW등급                                    \
EW등급                               관심      관찰1      관찰2      관찰3     부도   
0                   1차 금속 제조업  2059.0   5734.0   8692.0   8121.0  404.0   
1                   가구 내 고용활동     0.0      6.0      5.0     23.0    0.0   
2                      가구 제조업  1976.0   5782.0   7933.0   3042.0  119.0   
3             가죽, 가방 및 신발 제조업  1082.0   2436.0   4657.0   3014.0  101.0   
4               개인 및 소비용품 수리업  1314.0   2880.0   4889.0   1743.0    0.0   
..                        ...     ...      ...      ...      ...    ...   
75            하수, 폐수 및 분뇨 처리업   324.0   1146.0   1205.0    378.0    0.0   
76                     항공 운송업   109.0    293.0    358.0    233.0    0.0   
77                    협회 및 단체  3710.0  29269.0  26854.0   3989.0   12.0   
78    화학물질 및 화학제품 제조업; 의약품 제외  7939.0  16425.0  24394.0  12833.0  469.0   
79                환경 정화 및 복원업   126.0   3794.0   3350.0    389.0    0.0   

                                 
EW등급        정상       폐업      휴업  
0     140132.0  53432.0   738.0  
1        149.0     81.0     0.0  
2      83178.0  40085.0   394.0  
3      50162.0  26004.0   292.0  
4      91107.0  23388.0   301.0  
..         ...      ...     ...  
75     27594.0   6251.0    82.0  
76      4619.0   2481.0    10.0  
77    692671.0  31063.0   756.0  
78    331674.0  90419.0  1813.0  
79     28628.0  26529.0   282.0  

[80 rows x 9 columns]

In [12]:
# 합계 컬럼 추가
test1_1['총합']=test1_1.sum(axis=1)
#정상률
total1_1 = test1_1['총합']
normal1_1 = test1_1['EW등급']['정상']
rate1_1 = (normal1_1/total1_1)*100
test1_1['정상률']=rate1_1
#부도율 컬럼 추가
budo1_1=((test1_1['EW등급']['관심']*0.15)+(test1_1['EW등급']['관찰1']*0.20)+(test1_1['EW등급']['관찰2']*0.30)+(test1_1['EW등급']['관찰3']*0.90)+(test1_1['EW등급']['부도']*1))
budo_sum1_1=test1_1['EW등급']['관심']+test1_1['EW등급']['관찰1']+test1_1['EW등급']['관찰2']+test1_1['EW등급']['관찰3']+test1_1['EW등급']['부도']+test1_1['EW등급']['정상']+test1_1['EW등급']['폐업']
budo_rate1_1 = (budo1_1/budo_sum1_1)*100
test1_1['부도율']=budo_rate1_1
test1_1

업종중분류    EW등급                                    \
EW등급                               관심      관찰1      관찰2      관찰3     부도   
0                   1차 금속 제조업  2059.0   5734.0   8692.0   8121.0  404.0   
1                   가구 내 고용활동     0.0      6.0      5.0     23.0    0.0   
2                      가구 제조업  1976.0   5782.0   7933.0   3042.0  119.0   
3             가죽, 가방 및 신발 제조업  1082.0   2436.0   4657.0   3014.0  101.0   
4               개인 및 소비용품 수리업  1314.0   2880.0   4889.0   1743.0    0.0   
..                        ...     ...      ...      ...      ...    ...   
75            하수, 폐수 및 분뇨 처리업   324.0   1146.0   1205.0    378.0    0.0   
76                     항공 운송업   109.0    293.0    358.0    233.0    0.0   
77                    협회 및 단체  3710.0  29269.0  26854.0   3989.0   12.0   
78    화학물질 및 화학제품 제조업; 의약품 제외  7939.0  16425.0  24394.0  12833.0  469.0   
79                환경 정화 및 복원업   126.0   3794.0   3350.0    389.0    0.0   

                                       총합        정상률       부도율  
EW등급        정상       폐업      휴업                                 
0     140132.0  53432.0   738.0  219312.0  63.896184  5.387718  
1        149.0     81.0     0.0     264.0  56.439394  8.863636  
2      83178.0  40085.0   394.0  142509.0  58.366840  4.707103  
3      50162.0  26004.0   292.0   87748.0  57.165975  5.557309  
4      91107.0  23388.0   301.0  125622.0  72.524717  3.038996  
..         ...      ...     ...       ...        ...       ...  
75     27594.0   6251.0    82.0   36980.0  74.618713  2.654615  
76      4619.0   2481.0    10.0    8103.0  57.003579  4.844310  
77    692671.0  31063.0   756.0  788324.0  87.866283  2.294227  
78    331674.0  90419.0  1813.0  485966.0  68.250454  4.918435  
79     28628.0  26529.0   282.0   63098.0  45.370693  3.395313  

[80 rows x 12 columns]

In [13]:
b=test1_1.sort_values(by=test1_1.columns[-1],ascending=False).head(10)

In [14]:
b=b.set_index('업종중분류')
b

EW등급                                                           \
EW등급             관심     관찰1      관찰2     관찰3     부도       정상       폐업     휴업   
업종중분류                                                                          
운수 및 창고업        0.0    36.0      0.0     0.0    0.0      0.0      0.0    0.0   
광업 지원 서비스업    152.0  1632.0   2129.0   647.0    0.0   9405.0   2899.0   69.0   
가구 내 고용활동       0.0     6.0      5.0    23.0    0.0    149.0     81.0    0.0   
금속 광업          23.0   618.0    553.0   179.0    0.0   2965.0   1548.0   75.0   
기타 운송장비 제조업  2583.0  6231.0  14164.0  7448.0  352.0  93353.0  45759.0  673.0   
국제 및 외국기관      13.0   193.0    425.0    37.0    0.0   1703.0    288.0   17.0   
금융 및 보험업        0.0     0.0     24.0     0.0    0.0     84.0      0.0    0.0   
음료 제조업       1091.0  2061.0   2709.0  2075.0   56.0  32491.0   9255.0  180.0   
담배 제조업         15.0    31.0    169.0    64.0    0.0   1092.0    393.0   17.0   
수도업             4.0    85.0     67.0    52.0    0.0    942.0    139.0    0.0   

                   총합        정상률        부도율  
EW등급                                         
업종중분류                                        
운수 및 창고업         36.0   0.000000  20.000000  
광업 지원 서비스업    16933.0  55.542432   9.310958  
가구 내 고용활동       264.0  56.439394   8.863636  
금속 광업          5961.0  49.739977   7.714067  
기타 운송장비 제조업  170563.0  54.732269   7.615545  
국제 및 외국기관      2676.0  63.639761   7.572396  
금융 및 보험업        108.0  77.777778   6.666667  
음료 제조업        49918.0  65.088746   6.658993  
담배 제조업         1781.0  61.313869   6.618481  
수도업            1289.0  73.079907   6.555469

# 2. 전체연령대  업종비교 
- 10~90대까지의 연령대 데이터만을 사용

In [15]:
temp2= ~((df.대표자연령구간 == '10대미만') | (df.대표자연령구간 == '90대이상') | (df.대표자연령구간 == '정보없음'))
df2=df[temp2]

In [16]:
df2['기준일자']=df2['기준일자'].astype('str')
df2['기준일자']=pd.to_datetime(df2['기준일자'])
test2 = df2.groupby(['업종중분류','EW등급'])['EW등급'].count()
test2=pd.DataFrame(test2)

test2=test2.unstack()
test2 = test2.fillna(0)
test2=test2.reset_index()
# 합계 컬럼 추가
test2['총합']=test2.sum(axis=1)
#정상률
total2 = test2['총합']
normal2 = test2['EW등급']['정상']
rate2 = (normal2/total2)*100
test2['정상률']=rate2
#부도율 컬럼 추가
budo2=((test2['EW등급']['관심']*0.15)+(test2['EW등급']['관찰1']*0.20)+(test2['EW등급']['관찰2']*0.30)+(test2['EW등급']['관찰3']*0.90)+(test2['EW등급']['부도']*1))
budo_sum2=test2['EW등급']['관심']+test2['EW등급']['관찰1']+test2['EW등급']['관찰2']+test2['EW등급']['관찰3']+test2['EW등급']['부도']+test2['EW등급']['정상']+test2['EW등급']['폐업']
budo_rate2 = (budo2/budo_sum2)*100
test2['부도율']=budo_rate2
test2

<ipython-input-16-fdabd44e6a9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['기준일자']=df2['기준일자'].astype('str')
<ipython-input-16-fdabd44e6a9d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['기준일자']=pd.to_datetime(df2['기준일자'])


업종중분류    EW등급                                    \
EW등급                               관심      관찰1      관찰2      관찰3     부도   
0                   1차 금속 제조업  2548.0   6425.0  10084.0   9306.0  484.0   
1                   가구 내 고용활동     6.0     44.0     14.0     23.0    0.0   
2                      가구 제조업  2285.0   6245.0   8944.0   3725.0  147.0   
3             가죽, 가방 및 신발 제조업  1399.0   2797.0   5482.0   3675.0  106.0   
4               개인 및 소비용품 수리업  1583.0   3656.0   6604.0   2037.0    0.0   
..                        ...     ...      ...      ...      ...    ...   
76            하수, 폐수 및 분뇨 처리업   346.0   1322.0   1365.0    431.0    0.0   
77                     항공 운송업   110.0    300.0    441.0    243.0    0.0   
78                    협회 및 단체  4587.0  36538.0  32542.0   5019.0   36.0   
79    화학물질 및 화학제품 제조업; 의약품 제외  9942.0  19331.0  28800.0  14959.0  550.0   
80                환경 정화 및 복원업   132.0   4269.0   3613.0    397.0    0.0   

                                         총합        정상률       부도율  
EW등급        정상        폐업      휴업                                  
0     169942.0   63755.0   803.0   263347.0  64.531588  5.161725  
1        209.0      81.0     0.0      377.0  55.437666  9.177719  
2     101918.0   46385.0   420.0   170069.0  59.927441  4.582668  
3      65019.0   31771.0   387.0   110636.0  58.768394  5.285626  
4     106906.0   26492.0   350.0   147628.0  72.415802  3.247702  
..         ...       ...     ...        ...        ...       ...  
76     33262.0    6877.0   107.0    43710.0  76.097003  2.554182  
77      5454.0    2865.0    10.0     9423.0  57.879656  4.541591  
78    908720.0   38787.0   996.0  1027225.0  88.463579  2.174110  
79    418618.0  106897.0  2100.0   601197.0  69.630753  4.675470  
80     33627.0   30401.0   363.0    72802.0  46.189665  3.195516  

[81 rows x 12 columns]

In [17]:
test2.sort_values(by=test2.columns[-1],ascending=False).head(10)

업종중분류    EW등급                                    \
EW등급                                    관심      관찰1      관찰2      관찰3     부도   
50                        운수 및 창고업     0.0     36.0      0.0      0.0    0.0   
1                        가구 내 고용활동     6.0     44.0     14.0     23.0    0.0   
9                       광업 지원 서비스업   158.0   1836.0   2327.0    709.0    0.0   
19                     기타 운송장비 제조업  2982.0   7276.0  17516.0   8586.0  357.0   
12                           금속 광업    37.0    704.0    683.0    213.0    0.0   
15                        금융 및 보험업     0.0      0.0     24.0      0.0    0.0   
11                       국제 및 외국기관    13.0    193.0    437.0     37.0    0.0   
52                          음료 제조업  1377.0   2410.0   3473.0   2425.0   59.0   
66    전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업  9324.0  20422.0  54271.0  21518.0  231.0   
33                비금속광물 광업; 연료용 제외  1331.0   7280.0   9439.0   2307.0  123.0   

                                        총합        정상률        부도율  
EW등급        정상        폐업      휴업                                  
50         0.0       0.0     0.0      36.0   0.000000  20.000000  
1        209.0      81.0     0.0     377.0  55.437666   9.177719  
9      11738.0    3480.0    69.0   20317.0  57.774278   8.529731  
19    112383.0   54845.0   835.0  204780.0  54.879871   7.473436  
12      3943.0    1746.0    87.0    7413.0  53.190341   7.411275  
15        84.0       0.0     0.0     108.0  77.777778   6.666667  
11      2123.0     312.0    17.0    3132.0  67.784163   6.579454  
52     42842.0   11291.0   194.0   64071.0  66.866445   6.218122  
66    402335.0  190602.0  2699.0  701402.0  57.361542   5.919754  
33     71276.0   22674.0   430.0  114860.0  62.054675   5.843441

# 3. 지역별 2~30대 업종

In [19]:
temp3 = (df.대표자연령구간 == '20대') | (df.대표자연령구간 == '30대')
df3=df[temp3]

In [20]:
df3['기준일자']=df3['기준일자'].astype('str')
df3['기준일자']=pd.to_datetime(df3['기준일자'])
test3 = df3.groupby(['시도','업종중분류','EW등급'])['EW등급'].count()
test3=pd.DataFrame(test3)

test3=test3.unstack()
test3 = test3.fillna(0)
test3=test3.reset_index()
# 합계 컬럼 추가
test3['총합']=test3.sum(axis=1)
#정상률
total3 = test3['총합']
normal3 = test3['EW등급']['정상']
rate3 = (normal3/total3)*100
test3['정상률']=rate3
#부도율 컬럼 추가
budo3=((test3['EW등급']['관심']*0.15)+(test3['EW등급']['관찰1']*0.20)+(test3['EW등급']['관찰2']*0.30)+(test3['EW등급']['관찰3']*0.90)+(test3['EW등급']['부도']*1))
budo_sum3=test3['EW등급']['관심']+test3['EW등급']['관찰1']+test3['EW등급']['관찰2']+test3['EW등급']['관찰3']+test3['EW등급']['부도']+test3['EW등급']['정상']+test3['EW등급']['폐업']
budo_rate3 = (budo3/budo_sum3)*100
test3['부도율']=budo_rate3
test3

<ipython-input-20-508ffc91c732>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['기준일자']=df3['기준일자'].astype('str')
<ipython-input-20-508ffc91c732>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['기준일자']=pd.to_datetime(df3['기준일자'])


시도                       업종중분류  EW등급                                 \
EW등급                                    관심   관찰1   관찰2   관찰3   부도      정상   
0     강원                   1차 금속 제조업   3.0   0.0   1.0  18.0  0.0   144.0   
1     강원                      가구 제조업  14.0   1.0   0.0   0.0  0.0   126.0   
2     강원             가죽, 가방 및 신발 제조업   2.0   0.0   5.0  11.0  0.0    32.0   
3     강원               개인 및 소비용품 수리업   0.0  31.0   5.0  24.0  0.0   200.0   
4     강원  건축기술, 엔지니어링 및 기타 과학기술 서비스업  49.0  15.0   7.0   7.0  0.0  1223.0   
...   ..                         ...   ...   ...   ...   ...  ...     ...   
1155  충북     폐기물 수집, 운반, 처리 및 원료 재생업  25.0  24.0  65.0  68.0  0.0  1274.0   
1156  충북             하수, 폐수 및 분뇨 처리업   0.0   0.0   0.0   0.0  0.0    86.0   
1157  충북                     협회 및 단체  11.0  22.0  74.0   3.0  0.0   836.0   
1158  충북     화학물질 및 화학제품 제조업; 의약품 제외  56.0  10.0  65.0  19.0  0.0  2856.0   
1159  충북                 환경 정화 및 복원업   0.0  10.0  14.0   0.0  0.0    36.0   

                      총합         정상률        부도율  
EW등급     폐업   휴업                                 
0      36.0  0.0   202.0   71.287129   8.391089  
1       0.0  0.0   141.0   89.361702   1.631206  
2       0.0  0.0    50.0   64.000000  23.400000  
3       0.0  0.0   260.0   76.923077  11.269231  
4      51.0  0.0  1352.0   90.458580   1.386834  
...     ...  ...     ...         ...        ...  
1155   98.0  0.0  1554.0   81.981982   5.743243  
1156    0.0  0.0    86.0  100.000000   0.000000  
1157   31.0  0.0   977.0   85.568066   3.167861  
1158  214.0  1.0  3221.0   88.668115   1.459627  
1159    0.0  0.0    60.0   60.000000  10.333333  

[1160 rows x 13 columns]

In [21]:
c=test3.loc[test3.groupby(['시도'])['부도율'].idxmax()]
c=c.set_index('시도')
c

업종중분류  EW등급                                         \
EW등급                             관심   관찰1    관찰2   관찰3    부도     정상     폐업   
시도                                                                           
강원                        보건업   4.0   6.0   16.0   2.0  24.0   44.0    0.0   
경기                        보건업   5.0  50.0   89.0  73.0   0.0  520.0  165.0   
경남         공공행정, 국방 및 사회보장 행정   0.0   0.0   32.0   0.0   0.0   40.0    0.0   
경북            가죽, 가방 및 신발 제조업   0.0   3.0    2.0  30.0   0.0  109.0   36.0   
광주                     음료 제조업   0.0   0.0    0.0  33.0   0.0   93.0   40.0   
대구    컴퓨터 프로그래밍, 시스템 통합 및 관리업  13.0  74.0  165.0  68.0   0.0  746.0  294.0   
대전                  1차 금속 제조업  20.0   2.0   34.0  49.0   0.0  271.0   72.0   
부산                     음료 제조업  14.0  24.0    1.0  46.0   0.0  215.0    0.0   
서울                  가구 내 고용활동   0.0  27.0    9.0   0.0   0.0    0.0    0.0   
세종              개인 및 소비용품 수리업   0.0  12.0    0.0   0.0   0.0    0.0    0.0   
울산                        방송업   0.0   0.0   36.0   0.0   0.0    7.0    0.0   
인천                        수도업   0.0   3.0   20.0   0.0   0.0   13.0    0.0   
전남        영상ㆍ오디오 기록물 제작 및 배급업   0.0  22.0   37.0  42.0   0.0  177.0  107.0   
전북              인쇄 및 기록매체 복제업   0.0   4.0    2.0  27.0   0.0   99.0    4.0   
제주                기타 운송장비 제조업   0.0  10.0   24.0   0.0   0.0    2.0    0.0   
충남         공공행정, 국방 및 사회보장 행정   4.0   0.0    3.0   5.0   0.0   28.0    0.0   
충북                        보건업   8.0   0.0    2.0  55.0   0.0   34.0    0.0   

               총합        정상률        부도율  
EW등급   휴업                                
시도                                       
강원    0.0    96.0  45.833333  33.750000  
경기    0.0   902.0  57.649667  11.435698  
경남    0.0    72.0  55.555556  13.333333  
경북    3.0   183.0  59.562842  15.666667  
광주    2.0   168.0  55.357143  17.891566  
대구    0.0  1360.0  54.852941   9.371324  
대전    0.0   448.0  60.491071  12.879464  
부산    0.0   300.0  71.666667  16.200000  
서울    0.0    36.0   0.000000  22.500000  
세종    0.0    12.0   0.000000  20.000000  
울산    0.0    43.0  16.279070  25.116279  
인천    0.0    36.0  36.111111  18.333333  
전남    0.0   385.0  45.974026  13.844156  
전북    0.0   136.0  72.794118  18.897059  
제주    0.0    36.0   5.555556  25.555556  
충남    0.0    40.0  70.000000  15.000000  
충북    0.0    99.0  34.343434  51.818182

# 4. 지역별 전체연령 업종 분석

In [22]:
temp4= ~((df.대표자연령구간 == '10대미만') | (df.대표자연령구간 == '90대이상') | (df.대표자연령구간 == '정보없음'))
df4=df[temp4]

In [23]:
df4['기준일자']=df4['기준일자'].astype('str')
df4['기준일자']=pd.to_datetime(df4['기준일자'])
test4 = df4.groupby(['시도','업종중분류','EW등급'])['EW등급'].count()
test4=pd.DataFrame(test4)

test4=test4.unstack()
test4 = test4.fillna(0)
test4=test4.reset_index()
# 합계 컬럼 추가
test4['총합']=test4.sum(axis=1)
#정상률
total4 = test4['총합']
normal4 = test4['EW등급']['정상']
rate4 = (normal4/total4)*100
test4['정상률']=rate4
#부도율 컬럼 추가
budo4=((test4['EW등급']['관심']*0.15)+(test4['EW등급']['관찰1']*0.20)+(test4['EW등급']['관찰2']*0.30)+(test4['EW등급']['관찰3']*0.90)+(test4['EW등급']['부도']*1))
budo_sum4=test4['EW등급']['관심']+test4['EW등급']['관찰1']+test4['EW등급']['관찰2']+test4['EW등급']['관찰3']+test4['EW등급']['부도']+test4['EW등급']['정상']+test4['EW등급']['폐업']
budo_rate4 = (budo4/budo_sum4)*100
test4['부도율']=budo_rate4
test4

<ipython-input-23-459958fbc330>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['기준일자']=df4['기준일자'].astype('str')
<ipython-input-23-459958fbc330>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['기준일자']=pd.to_datetime(df4['기준일자'])


시도                       업종중분류   EW등급                              \
EW등급                                     관심    관찰1     관찰2    관찰3    부도   
0     강원                   1차 금속 제조업   34.0   68.0    32.0  140.0   0.0   
1     강원                      가구 제조업   54.0   63.0   170.0  110.0  28.0   
2     강원             가죽, 가방 및 신발 제조업    5.0   73.0    37.0   41.0   0.0   
3     강원               개인 및 소비용품 수리업   30.0  155.0   141.0   71.0   0.0   
4     강원  건축기술, 엔지니어링 및 기타 과학기술 서비스업  204.0  465.0   471.0   77.0   0.0   
...   ..                         ...    ...    ...     ...    ...   ...   
1255  충북             하수, 폐수 및 분뇨 처리업    0.0   30.0    83.0    0.0   0.0   
1256  충북                      항공 운송업    0.0    8.0    25.0    0.0   0.0   
1257  충북                     협회 및 단체  125.0  882.0   954.0  114.0   0.0   
1258  충북     화학물질 및 화학제품 제조업; 의약품 제외  546.0  659.0  1370.0  865.0  26.0   
1259  충북                 환경 정화 및 복원업    0.0   37.0    19.0    0.0   0.0   

                                  총합        정상률        부도율  
EW등급       정상      폐업    휴업                                 
0      1788.0   635.0  10.0   2707.0  66.050979   5.721172  
1      1415.0   438.0   6.0   2284.0  61.952715   8.722564  
2       134.0   144.0   0.0    434.0  30.875576  14.596774  
3      2469.0   665.0   1.0   3532.0  69.903737   4.013027  
4     14188.0  2033.0  72.0  17510.0  81.027984   1.916504  
...       ...     ...   ...      ...        ...        ...  
1255   1019.0   233.0   0.0   1365.0  74.652015   2.263736  
1256     77.0     0.0   0.0    110.0  70.000000   8.272727  
1257  26904.0  1175.0  44.0  30198.0  89.091993   1.936559  
1258  23475.0  5399.0  37.0  32377.0  72.505173   4.419295  
1259    266.0   108.0   0.0    430.0  61.860465   3.046512  

[1260 rows x 13 columns]

In [24]:
d=test4.loc[test4.groupby(['시도'])['부도율'].idxmax()]
d=d.set_index('시도')
d

업종중분류  EW등급                                            \
EW등급                            관심    관찰1    관찰2    관찰3    부도      정상     폐업   
시도                                                                             
강원                    담배 제조업   0.0    0.0   18.0   18.0   0.0     0.0    0.0   
경기                 가구 내 고용활동   0.0    6.0    4.0   17.0   0.0    45.0   72.0   
경남                광업 지원 서비스업   6.0   85.0   63.0   36.0   0.0   482.0   54.0   
경북                       수도업   0.0    0.0   10.0   26.0   0.0    72.0   36.0   
광주                 국제 및 외국기관   0.0    0.0   60.0    0.0   0.0    36.0    0.0   
대구                        어업   0.0    1.0   36.0   24.0   0.0   173.0   36.0   
대전                        어업   9.0   60.0   69.0   14.0   0.0   375.0   51.0   
부산                    음료 제조업  35.0  112.0   71.0  233.0  10.0  1198.0  321.0   
서울                  운수 및 창고업   0.0   36.0    0.0    0.0   0.0     0.0    0.0   
세종    의복, 의복 액세서리 및 모피제품 제조업  22.0   13.0   44.0   25.0   0.0   133.0    0.0   
울산                       방송업   0.0    2.0  104.0    0.0   0.0   149.0   40.0   
인천                 가구 내 고용활동   6.0    0.0    1.0    6.0   0.0    23.0    0.0   
전남                    항공 운송업  10.0    5.0    1.0   26.0   0.0   143.0    9.0   
전북                광업 지원 서비스업   6.0   26.0   75.0   96.0   0.0   489.0  123.0   
제주                    항공 운송업   6.0    0.0    1.0   35.0   0.0   102.0  108.0   
충남                광업 지원 서비스업   6.0  131.0   44.0   57.0   0.0   360.0   13.0   
충북          석탄, 원유 및 천연가스 광업   0.0    4.0   30.0   13.0   0.0    69.0   36.0   

               총합        정상률        부도율  
EW등급   휴업                                
시도                                       
강원    0.0    36.0   0.000000  60.000000  
경기    0.0   144.0  31.250000  12.291667  
경남    0.0   726.0  66.391185   9.531680  
경북    0.0   144.0  50.000000  18.333333  
광주    0.0    96.0  37.500000  18.750000  
대구    0.0   270.0  64.074074  12.074074  
대전    0.0   578.0  64.878893   8.070934  
부산    2.0  1982.0  60.443996  13.568182  
서울    0.0    36.0   0.000000  20.000000  
세종    0.0   237.0  56.118143  17.552743  
울산    0.0   295.0  50.508475  10.711864  
인천    0.0    36.0  63.888889  18.333333  
전남    0.0   194.0  73.711340  13.505155  
전북    0.0   815.0  60.000000  14.110429  
제주    0.0   252.0  40.476190  12.976190  
충남    0.0   611.0  58.919804  14.991817  
충북    0.0   152.0  45.394737  14.144737